In [1]:
import pandas as pd
import catboost
import numpy as np
import tqdm
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, FeaturesData, Pool
from matplotlib import pyplot as plt

In [2]:
best_combinations_amount = 10
data_train = pd.read_csv("input_5_point.csv", engine='python', encoding="windows-1251", sep=";", index_col=False,
                             skiprows=2)

nns = data_train.loc[data_train["Well_type"] == 1]
nns_grp = data_train.loc[data_train["Well_type"] == 2]
gs = data_train.loc[data_train["Well_type"] == 3]
gs_grp = data_train.loc[data_train["Well_type"] == 4]

In [3]:
nns_numeric_features_names = ["PSS", "Pprod", "Pinj", "Perm", "h", "a/b", "Poro", "Sw", "VdP", "Pbp", "mo",
                                  "Bo", "po", "mw", "Bw", "Pres"]
nns_features = nns[nns_numeric_features_names]
nns_target = nns[["NPV"]]

nns_grp_numeric_features_names = ["PSS", "Pprod", "Pinj", "Perm", "h", "a/b", "Poro", "Sw", "VdP", "Pbp", "mo",
                                  "Bo", "po", "mw", "Bw", "Pres",
                                  "Lfracprod", "Lfracinj", "Frac_perm", "Wprod", "Winj"]
nns_grp_features = nns_grp[nns_grp_numeric_features_names]
nns_grp_target = nns_grp[["NPV"]]

gs_numeric_features_names = ["PSS", "Pprod", "Pinj", "Perm", "h", "a/b", "Poro", "Sw", "VdP", "Pbp", "mo",
                              "Bo", "po", "mw", "Bw", "Pres", "Lgsprod", "Lgsinj"]
gs_features = gs[gs_numeric_features_names]
gs_target = gs[["NPV"]]

gs_grp_numeric_features_names = ["PSS", "Pprod", "Pinj", "Perm", "h", "a/b", "Poro", "Sw", "VdP", "Pbp", "mo",
                                  "Bo", "po", "mw", "Bw", "Pres",
                                 "Lfracprod", "Lfracinj", "Frac_perm", "Wprod", "Winj",
                                 "Lgsprod", "Lgsinj", "Nfracprod", "Nfracinj"]
gs_grp_features = gs_grp[gs_grp_numeric_features_names]
gs_grp_target = gs_grp[["NPV"]]

In [4]:
nns_model = CatBoostRegressor()
nns_model.load_model("5_point_full_trained_catboosts/nns_5_point_model", format = 'json')
nns_npvs = pd.DataFrame(np.column_stack((nns_model.predict(nns_features), np.array(nns_target)[:,0])))

nns_grp_model = CatBoostRegressor()
nns_grp_model.load_model("5_point_full_trained_catboosts/nns_grp_5_point_model", format = 'json')
nns_grp_npvs = pd.DataFrame(np.column_stack((nns_grp_model.predict(nns_grp_features), np.array(nns_grp_target)[:,0])))

gs_model = CatBoostRegressor()
gs_model.load_model("5_point_full_trained_catboosts/gs_5_point_model", format = 'json')
gs_npvs = pd.DataFrame(np.column_stack((gs_model.predict(gs_features), np.array(gs_target)[:,0])))

gs_grp_model = CatBoostRegressor()
gs_grp_model.load_model("5_point_full_trained_catboosts/gs_grp_5_point_model", format = 'json')
gs_grp_npvs = pd.DataFrame(np.column_stack((gs_grp_model.predict(gs_grp_features), np.array(gs_grp_target)[:,0])))

In [5]:
npvs = pd.concat([nns_npvs, nns_grp_npvs, gs_npvs, gs_grp_npvs])

best_npv_combination_NumEX = np.sort(((npvs.sort_values([0], ascending=False))[0:best_combinations_amount]).index)
best_npv_combination_ML = np.sort(((npvs.sort_values([1], ascending=False))[0:best_combinations_amount]).index)

common_best_combinations = set(best_npv_combination_NumEX).intersection(set(best_npv_combination_ML))
print("{0} from {1}".format(len(common_best_combinations), best_combinations_amount)) 

9 from 10
